In [1]:
%matplotlib inline
import re

# Finding Evidence Workshop

## DIY Text Mining Session

### 1. A Simple dictionary based text mining

A dcitionary is a collection of terms.

In [ ]:
sample_dictionary = {'cancer', 'diabete', 'asthma'}

We use a dictionary to find out whether a term appear in a give context.

In [16]:
sample_text = 'Cancer, diabete and asthma are diseases. Thanks for support from Cancer Research Organization.'

We need a piece of magic to find the terms from a context.

In [17]:
def find_evidence_sample(dictionary, text):
    """
    given a dictionary and a piece of text, find the evidence from the text
    """
    for d in dictionary:
        pattern = re.compile(re.escape(d))
        matches = pattern.finditer(text)
        if matches:
            for m in matches:
                print(m)

Let's run the codes and see what happens.

In [18]:
find_evidence_sample(sample_dictionary, sample_text)

<_sre.SRE_Match object; span=(8, 15), match='diabete'>
<_sre.SRE_Match object; span=(20, 26), match='asthma'>


"Cancer" with a capital 'C' is missing. Let's convert all the words to lowercase.

In [19]:
find_evidence_sample(sample_dictionary, sample_text.lower())

<_sre.SRE_Match object; span=(8, 15), match='diabete'>
<_sre.SRE_Match object; span=(20, 26), match='asthma'>
<_sre.SRE_Match object; span=(0, 6), match='cancer'>
<_sre.SRE_Match object; span=(65, 71), match='cancer'>


All the "cancer" in the text are identified, however, we don't want the "Cancer" in "Cancer Research Organization" to be identified. What can we do? Let's refine our dictionary.

In [40]:
sample_dictionary_refined = {'cancer': {'exclude': {'post':{'organization'}}}, 'diabete':{}, 'asthma':{}}

In [54]:
def check_exclude(dictionary, text):
    if 'post' in dictionary:
        for exclude in dictionary['post']:
            exclude_pattern = re.compile(re.escape(exclude))
            print(exclude, text)
            if exclude_pattern.search(text):
                return True
            else:
                return False
    
def find_evidence_refined(dictionary, text, chars_len=30):
    evidences = []
    for d in dictionary:
        pattern = re.compile(re.escape(d))
        matches = pattern.finditer(text)
        if matches:
            for m in matches:
                start,end = m.span(0)
                print(m)
                if 'exclude' in dictionary[d]:
                    exclude = check_exclude(dictionary[d]['exclude'], text[end:end+chars_len])
                    if exclude:
                        continue
                evidences.append(m)
    print(evidences)


In [55]:
find_evidence_refined(sample_dictionary_refined, sample_text.lower())

<_sre.SRE_Match object; span=(0, 6), match='cancer'>
organization , diabete and asthma are disea
<_sre.SRE_Match object; span=(65, 71), match='cancer'>
organization  research organization.
<_sre.SRE_Match object; span=(8, 15), match='diabete'>
<_sre.SRE_Match object; span=(20, 26), match='asthma'>
[<_sre.SRE_Match object; span=(0, 6), match='cancer'>, <_sre.SRE_Match object; span=(8, 15), match='diabete'>, <_sre.SRE_Match object; span=(20, 26), match='asthma'>]


### Improved version